In [1]:
import numpy as np
import random
from platypus import Hypervolume, display, calculate, CMAES, MOEAD, NSGAII, NSGAIII, SPEA2, IBEA, Problem, Real
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

from datetime import datetime

In [2]:
from load_data import *
from bias_functions import *
from utility_functions import *

In [3]:
def getDataset(setSelection):
    if setSelection == 'toy':
        X,y = generate_toy_data(1000,200,2)
    elif setSelection == 'adult':
        protectedAttributes={'race':'White','gender':'Male'}
        X,y = load_adult(protectedAttributes=protectedAttributes)
    elif setSelection == 'bank':
        X,y = load_bank()
    elif setSelection == 'german':
        X,y = load_german()
    elif setSelection == 'mortgage':
        protectedCategoricalAttributes={'applicant_ethnicity_name':'Not Hispanic or Latino',
                                'applicant_race_name_1':'White','applicant_sex_name':'Male'}
        protectedNumericalAttributes=['minority_population']
        X,y = load_mortgage(protectedCategoricalAttributes=protectedCategoricalAttributes, \
                            protectedNumericalAttributes=protectedNumericalAttributes)
    else:
        print('dataset not recognised')
        
    X = np.hstack([X, np.ones((X.shape[0],1))]) ## add ones to solve for affine functions
    
    return X,y

In [6]:
# existing problems

# problem 1: quadriato, adult dataset, sensistive attribute = gender (9), train/test = 0.8/0.2
# objectives: accuracy, DM(OMR)

def problem1_base(w,X,y,sensitiveAttributeIndex):
    errorRateObjective = errorRate(w,X,y)
    fairnessObjective= differenceDisparateMistreatment(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex,type='OMR')
    return errorRateObjective, np.abs(fairnessObjective[0]-fairnessObjective[1])

def problem1_train(w):
    return problem1_base(w,trainxs,trainys,sensitiveAttributeIndex=9)
def problem1_test(w):
    return problem1_base(w,testxs,testys,sensitiveAttributeIndex=9)

# problem 1 (amended): quadriato, adult dataset, sensistive attribute = gender (9), train/test = 0.8/0.2
# objectives: accuracy, DM(OMR), DM(FPR), DM(FNR)

def problem1amended_base(w,X,y,sensitiveAttributeIndex):
    errorRateObjective = errorRate(w,X,y)
    fairnessObjective1 = differenceDisparateMistreatment(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex,type='OMR')
    fairnessObjective2 = differenceDisparateMistreatment(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex,type='FNR')
    fairnessObjective3 = differenceDisparateMistreatment(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex,type='FPR')
    return errorRateObjective, np.abs(fairnessObjective1[0]-fairnessObjective1[1]), np.abs(fairnessObjective2[0]-fairnessObjective2[1]), np.abs(fairnessObjective3[0]-fairnessObjective3[1])

def problem1amended_train(w):
    return problem1amended_base(w,trainxs,trainys,sensitiveAttributeIndex=9)
def problem1amended_test(w):
    return problem1amended_base(w,testxs,testys,sensitiveAttributeIndex=9)

# problem 2: zafar, bank dataset, sensistive attribute = age (0), train/test = 0.7/0.3
# objectives: accuracy, DI

def problem2_base(w,X,y,sensitiveAttributeIndex):
    errorRateObjective = errorRate(w,X,y)
    fairnessObjective= differenceDisparateImpactModel(w,X,sensitiveAttributeIndex=sensitiveAttributeIndex)
    return errorRateObjective, np.abs(fairnessObjective[0]-fairnessObjective[1])

def problem2_train(w):
    return problem2_base(w,trainxs,trainys,sensitiveAttributeIndex=0)
def problem2_test(w):
    return problem2_base(w,testxs,testys,sensitiveAttributeIndex=0)

# problem 3: donini, german dataset, sensistive attribute = race (14), train/test = 1.0/0.0
# objectives: accuracy, DI

def problem3_base(w,X,y,sensitiveAttributeIndex):
    errorRateObjective = errorRate(w,X,y)
    fairnessObjective= differenceEqualOpportunity(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex)
    return errorRateObjective, np.abs(fairnessObjective[0]-fairnessObjective[1])

def problem3_train(w):
    return problem3_base(w,trainxs,trainys,sensitiveAttributeIndex=14)
def problem3_test(w):
    return problem3_base(w,testxs,testys,sensitiveAttributeIndex=14)

## Sensitive attributes list:

adult: 8 - race, 9 - gender ;  variables = 15 ; 45222 samples

bank: 0 - age ; variables = 21 ; 41188 samples

german: 6 - gender, 9 - age, 14 - foreign worker ; variables = 25 ; 1000 samples

mortgage: 12 - ethnicity, 14 - race, 16 - gender, 24 - minority population ; variables = 30 ; 200000 samples

In [11]:
now = datetime.now()

# these lines define the problem, test problem, dataset, algorithm and number of sampling runs
noOfVariables = 15
noOfObjectives = 2

problem = Problem(noOfVariables,noOfObjectives)
problem.function = problem1_train

testProblem = Problem(noOfVariables,noOfObjectives)
testProblem.function = problem1_test

dataset = 'adult'
trainAlgorithm = CMAES(problem)
# trainAlgorithm = MOEAD(problem)
# trainAlgorithm = NSGAII(problem)
# trainAlgorithm = NSGAIII(problem, divisions_outer=96)
# trainAlgorithm = SPEA2(problem)
# trainAlgorithm = IBEA(problem)

noOfSamplingRuns = 1

X,y = getDataset(dataset)

problem.types[:] = Real(-5,5)

hypervolumeArray = np.zeros((noOfSamplingRuns, 2)) # col 0 for training results (hypervolume), col 1 for test results
extremesArray = np.zeros((2, noOfObjectives, noOfObjectives, noOfSamplingRuns)) # the first dimension is for train/test
averagePointArray = np.zeros((2, noOfSamplingRuns, noOfObjectives)) # the first dimension is for train/test. This array holds the average objective point for each run

# these will be used to store the algorithm that returns the largest hypervolume
bestTrainAlgorithm = None
bestTrainHypervolume = 0
bestTestAlgorithm = None
bestTestHypervolume = 0
totalTrainTime = 0
totalTestTime = 0

for run in range(noOfSamplingRuns):

    # generate new train/test split for each run
    trainxs, testxs, trainys, testys  = train_test_split(X,y,train_size=0.8)

    # for German dataset only
#     trainxs = X
#     trainys = y
#     testxs = X
#     testys = y
    
    
    # training run
    startTime = time.perf_counter()
    trainAlgorithm.run(10)

    trainHyp = Hypervolume(minimum=[0]*noOfObjectives,maximum=[1]*noOfObjectives)
    trainHypResult = trainHyp(trainAlgorithm.result)  
    hypervolumeArray[run, 0] = trainHypResult
    
    singleRunExtremesArray, averagePoint = findExtremes_averagePoint(trainAlgorithm)
    extremesArray[0,:,:,run] = singleRunExtremesArray
    averagePointArray[0,run,:] = averagePoint
    
    endTime = time.perf_counter()
    trainTime = endTime - startTime
    totalTrainTime += trainTime
    
    print("{}: training took {:.2f} seconds w/ hypervolume {:2f}".format(run, trainTime, trainHypResult))
    
    
    # test run
    testAlgorithm = deepcopy(trainAlgorithm)

    startTime = time.perf_counter()

    for resultNumber, result in enumerate(testAlgorithm.result):

        w=result.variables
        objectivesResult = testProblem.function(w)

        # copy objective results individually to the testAlgorithm object
        for i in range(len(objectivesResult)):  
            testAlgorithm.result[resultNumber].objectives[i]=objectivesResult[i]

    testHyp = Hypervolume(minimum=[0]*noOfObjectives,maximum=[1]*noOfObjectives)
    testHypResult = testHyp(testAlgorithm.result)
    hypervolumeArray[run, 1] = testHypResult
    
    singleRunExtremesArray, averagePoint = findExtremes_averagePoint(testAlgorithm)
    extremesArray[1,:,:,run] = singleRunExtremesArray
    averagePointArray[1,run,:] = averagePoint
    
    endTime = time.perf_counter()
    testTime = endTime - startTime
    totalTestTime += testTime

    print("{}: testing took {:.2f} seconds w/ hypervolume {:2f}".format(run, testTime, testHypResult))
    
    if trainHypResult > bestTrainHypervolume:
        bestTrainHypervolume = trainHypResult
        bestTrainAlgorithm = deepcopy(trainAlgorithm)
        
        bestTestHypervolume = testHypResult
        bestTestAlgorithm = deepcopy(testAlgorithm)
        
#         bestData = [trainxs, testxs, trainys, testys]
        bestData = [trainxs, testxs]
        bestDataSavePath = "saved_data/"+now.strftime("%Y%m%d_%H%M")+"_"+str(problem.function)[10:18]+"_"+dataset+"_"+str(trainAlgorithm)[21:-26]+"_data"
        np.save(bestDataSavePath, bestData)

# resultsSummary:
# column 0: average hypervolume, column 1: hypervolume std, column 2: average point std (Euclidean distance), column 3: average time
# row 0: training data, row 1: test data
resultsSummary = np.zeros((2,4)) 
resultsMean = np.sum(hypervolumeArray, axis=0)/noOfSamplingRuns 
resultsSummary[:,0] = resultsMean
resultsSummary[:,1] = np.sqrt(np.sum((hypervolumeArray - resultsMean)**2, axis=0)/noOfSamplingRuns)
averagePointMeans = np.mean(averagePointArray,axis=1)
resultsSummary[0,2] = np.std(np.sqrt(np.sum((averagePointArray[0,:,:]-averagePointMeans[0,:])**2,axis=1)))
resultsSummary[1,2] = np.std(np.sqrt(np.sum((averagePointArray[1,:,:]-averagePointMeans[1,:])**2,axis=1)))
resultsSummary[0,3] = totalTrainTime/noOfSamplingRuns
resultsSummary[1,3] = totalTestTime/noOfSamplingRuns
print(resultsSummary)



training: 32561
testing: 16281
(45222, 15)
(45222, 14)
race: 8
gender: 9
The dataset is loaded...
0: training took 0.31 seconds w/ hypervolume 0.725201
0: testing took 0.07 seconds w/ hypervolume 0.730425
[[0.72520074 0.         0.         0.31281376]
 [0.73042529 0.         0.         0.07196009]]


/Users/nigelkingsman/opt/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
